In [1]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

# Upload kaggle API key file
uploaded = files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v
!kaggle datasets download -d paultimothymooney/breast-histopathology-images


Saving kaggle.json to kaggle.json
kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 6.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=1dd37c610c72929c4b37a479e4c2e10ff113b22356d8683ec18629102f3b34cf
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Kaggle API 1.5.6
100% 3.10G/3.10G [00:49<00:00, 104MB/s] 
100% 3.10G/3.10G [00:49<00:00, 66.5MB/s]


In [0]:
!unzip \*.zip 

In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import fnmatch
from glob import glob
%matplotlib inline 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import keras
import tensorflow
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPool2D,MaxPooling2D
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

Using TensorFlow backend.


In [0]:
from glob import glob
images=glob('../content/IDC_regular_ps50_idx5/**/*.png',recursive=True)

In [0]:
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(images, patternZero)
classOne = fnmatch.filter(images, patternOne)

In [5]:
import numpy as np
def process_images(lowerIndex,upperIndex):
    x = []
    y = []
    WIDTH = 50
    HEIGHT = 50
    count = 0
    for img in images[lowerIndex:upperIndex]:
        full_size_image = cv2.imread(img)
        x.append(cv2.resize(full_size_image, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))
        if img in classZero:
            y.append(0)
        elif img in classOne:
            y.append(1)
        else:
            return
        if(count%10000==0):
            print(count)
        count +=1
    return x,y

X,Y=process_images(0,150000)

X1=np.array(X)
print(X1.shape)
df = pd.DataFrame()
df["images"]=X
df["labels"]=Y
X2=df["images"]
Y2=df["labels"]
X2=np.array(X2)

# Separation of classes of images
imgs0=[]
imgs1=[]
imgs0 = X2[Y2==0]
imgs1 = X2[Y2==1] 

#print number of elements in eachi class label
print('Total number of images: {}'.format(len(X2)))
print('Number of Class 0 images: {}'.format(np.sum(Y2==0)))
print('Number of class 1 Images: {}'.format(np.sum(Y2==1)))
print('Image shape (Width, Height, Channels): {}'.format(X1[0].shape))

X=np.array(X)

#Standarizing the data for the model 
X=X/255.0

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
(150000, 50, 50, 3)
Total number of images: 150000
Number of Class 0 images: 106919
Number of class 1 Images: 43081
Image shape (Width, Height, Channels): (50, 50, 3)


In [0]:
y_trainCat=to_categorical(Y_train,num_classes=2)
y_testCat=to_categorical(Y_test,num_classes=2)

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

In [0]:
batch_size = 128
num_classes = 2
epochs = 10
img_rows,img_cols=50,50
input_shape = (img_rows, img_cols, 3)
e = 2

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(4, 4),activation='relu',input_shape=input_shape,strides=e))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [14]:
history = model.fit_generator(datagen.flow(X_train,y_trainCat, batch_size=32),
                        steps_per_epoch=len(X_train) / 32,epochs=10,validation_data = [X_test, y_testCat])

Epoch 1/10
3750/3750 [==============================] - 124s 33ms/step - loss: 0.4200 - accuracy: 0.8171 - val_loss: 0.3711 - val_accuracy: 0.8384
Epoch 2/10
3750/3750 [==============================] - 119s 32ms/step - loss: 0.3830 - accuracy: 0.8351 - val_loss: 0.3595 - val_accuracy: 0.8462
Epoch 3/10
3750/3750 [==============================] - 118s 31ms/step - loss: 0.3688 - accuracy: 0.8417 - val_loss: 0.3810 - val_accuracy: 0.8353
Epoch 4/10
3750/3750 [==============================] - 117s 31ms/step - loss: 0.3595 - accuracy: 0.8468 - val_loss: 0.3489 - val_accuracy: 0.8549
Epoch 5/10
3750/3750 [==============================] - 120s 32ms/step - loss: 0.3531 - accuracy: 0.8498 - val_loss: 0.3421 - val_accuracy: 0.8524
Epoch 6/10
3750/3750 [==============================] - 122s 32ms/step - loss: 0.3486 - accuracy: 0.8520 - val_loss: 0.3767 - val_accuracy: 0.8349
Epoch 7/10
3750/3750 [==============================] - 120s 32ms/step - loss: 0.3441 - accuracy: 0.8542 - val_loss: 0